In [37]:
from pathlib import Path
from datetime import datetime
import os
import pandas as pd

In [38]:
def drop_duplicate_columns(df):
    """ Drop duplicate columns from a DataFrame, keeping the first one encountered.
    
    Args:
        df (pd.DataFrame): Input DataFrame from which to remove duplicate columns.
        
    Returns:
        pd.DataFrame: DataFrame with duplicate columns removed.
    """
    # Transpose the DataFrame, drop duplicates, then transpose back to original form
    return df.T.drop_duplicates().T

def conc_dfs(data_dict:dict):
    df = pd.concat(data_dict.values(), axis=0)
    df = df.fillna(0)
    df = df.infer_objects()
    df = drop_duplicate_columns(df)
    return df

In [39]:
dateStr = datetime.date(datetime.now()).strftime("%d_%m_%Y")
# folder_to_processed = Path(r'C:\Users\candr\Documents\masters\SecurityAnalysisDNN\src\SecurityAnalysisDNN\data\processedData')
folder_to_processed = Path(r'C:\Users\candr\Documents\masters\SecurityAnalysisDNN\src\SecurityAnalysisDNN\data\processedData\AAPL\12_05_2024')

data_train_dict = {}
data_val_dict = {}
for file_path in folder_to_processed.rglob('*.xls*'):
    if dateStr in file_path.name:
        symbol = file_path.name.split('_')[0]
        data_df = pd.read_excel(file_path, index_col='Unnamed: 0')

        n_train_rows = len(data_df) * 4 // 5
        data_train_dict[symbol] = data_df.iloc[:n_train_rows]
        data_val_dict[symbol] = data_df.iloc[n_train_rows:]

data_train_df = conc_dfs(data_train_dict)
data_val_df = conc_dfs(data_val_dict)

common_columns = data_train_df.columns.intersection(data_val_df.columns)
data_train_df = data_train_df[common_columns]
data_val_df = data_val_df[common_columns]

In [41]:
data_train_df.shape, data_val_df.shape

((98, 101), (25, 101))

In [42]:
columns_to_keep = [
    # 'Revenue',
    # 'Revenue Growth',
    # 'Cost of Revenue',
    'Gross Profit',
    # 'Selling, General & Admin',
    'Research & Development',
    # 'Operating Expenses',
    # 'Other Operating Expenses',
    'Operating Income',
    # 'Interest Expense / Income',
    # 'Other Expense / Income',
    # 'Pretax Income',
    # 'Income Tax',
    'Net Income',
    # 'Preferred Dividends',
    # 'Net Income Common',
    # 'Net Income Growth',
    'Shares Outstanding (Basic)',
    # 'Shares Outstanding (Diluted)',
    # 'Shares Change',
    # 'EPS (Basic)',
    # 'EPS (Diluted)',
    # 'EPS Growth',
    'Free Cash Flow Per Share',
    'Dividend Per Share',
    # 'Dividend Growth',
    # 'Gross Margin',
    # 'Operating Margin',
    'Profit Margin',
    # 'Free Cash Flow Margin',
    # 'Effective Tax Rate',
    'EBITDA',
    # 'EBITDA Margin',
    # 'Depreciation & Amortization',
    # 'EBIT',
    # 'EBIT Margin',

    # ? Balance Sheet
    # 'Cash & Equivalents',
    # 'Short-TermInvestments',
    'Cash & Cash Equivalents',
    # 'Cash Growth',
    # 'Receivables',
    # 'Inventory',
    # 'Other Current Assets',
    'Total Current Assets',
    # 'Property, Plant & Equipment',
    # 'Long-Term Investments',
    # 'Goodwill and Intangibles',
    # 'Other Long-Term Assets',
    # 'Total Long-Term Assets',
    'Total Assets',
    # 'Accounts Payable',
    # 'Deferred Revenue',
    # 'Current Debt',
    'Total Current Liabilities',
    # 'Other Current Liabilities',
    # 'Long-Term Debt',
    'Total Long-Term Liabilities',
    # 'Other Long-Term Liabilities',
    'Total Liabilities',
    'Total Debt',
    # 'Debt Growth',
    # 'Retained Earnings',
    # 'Comprehensive Income',
    'Shareholders Equity',
    # 'Net Cash / Debt',
    # 'Net Cash / Debt Growth',
    'Net Cash Per Share',
    'Working Capital',
    'Book Value Per Share',

    # ? Cash Flow Statement
    # 'Share-Based Compensation',
    # 'Other Operating Activities',
    'Operating Cash Flow',
    # 'Operating Cash Flow Growth',
    # 'Capital Expenditures',
    # 'Acquisitions',
    # 'Change in Investments',
    # 'Other Investing Activities',
    'Investing Cash Flow',
    # 'Dividends Paid',
    # 'Share Issuance / Repurchase',
    # 'Debt Issued / Paid',
    # 'Other Financing Activities',
    'Financing Cash Flow',
    # 'Net Cash Flow',
    # 'Free Cash Flow',
    # 'Free Cash Flow Growth',

    # ? Ratios and Metrics
    # 'Market Capitalization',
    # 'Market Cap Growth',
    # 'Enterprise Value',
    # 'PE Ratio',
    # 'PS Ratio',
    # 'PB Ratio',
    # 'P/FCF Ratio',
    # 'P/OCF Ratio',
    'Debt/Equity',
    'Quick Ratio',
    'Current Ratio',
    # 'Asset Turnover',
    # 'Return on Invested Capital (ROIC)',
    'Dividend Yield',
    'Payout Ratio',
    # 'Buyback Yield',
    'Total Return',
    'Close Price']

shares_outstanding_col = 'Shares Outstanding (Basic)'

columns_to_normalize_per_share = [
    'Gross Profit',
    'Research & Development',
    'Operating Income',
    'Net Income',
    # 'Shares Outstanding (Basic)',
    # 'Free Cash Flow Per Share',
    # 'Dividend Per Share',
    # 'Profit Margin',
    'EBITDA',
    'Cash & Cash Equivalents',
    'Total Current Assets',
    'Total Assets',
    'Total Current Liabilities',
    'Total Long-Term Liabilities',
    'Total Liabilities',
    'Total Debt',
    'Shareholders Equity',
    # 'Net Cash Per Share',
    'Working Capital',
    # 'Book Value Per Share',
    'Operating Cash Flow',
    'Investing Cash Flow',
    'Financing Cash Flow',
    # 'Enterprise Value',
    # 'Debt/Equity',
    # 'Quick Ratio',
    # 'Current Ratio',
    # 'Dividend Yield',
    # 'Payout Ratio',
    # 'Total Return',
    # 'Close Price'
]

In [43]:
def reduce_df_standardize(df:pd.DataFrame, cols_to_keep:list, num_shares_col:str):
    try:
        df = df.loc[:, cols_to_keep]
    except KeyError:
        cols_to_keep.remove('Cash & Cash Equivalents')
        cols_to_keep.append('Cash & Equivalents')

        columns_to_normalize_per_share.remove('Cash & Cash Equivalents')
        columns_to_normalize_per_share.append('Cash & Equivalents')

        df = df.loc[:, cols_to_keep]

    # todo move the index to 'Date' column then reset it.
    df.reset_index(inplace=True)

    # Filter rows where the num_shares_col is not equal to zero
    df = df[df[num_shares_col] != 0]

    shares_outstanding = df[num_shares_col]
    if [(idx, value) for idx, value in enumerate(shares_outstanding.values) if not value]:
        raise Exception("Can't have 0 shares outstanding.")

    # Get the per share values
    df[columns_to_normalize_per_share] = df[columns_to_normalize_per_share].div(shares_outstanding, axis=0)
    df = df.drop(columns=[num_shares_col])
    return df

In [44]:
reduced_data_train_df = reduce_df_standardize(data_train_df.copy(), columns_to_keep, shares_outstanding_col)
reduced_data_train_df.head(3)

,index,Gross Profit,Research & Development,Operating Income,Net Income,Free Cash Flow Per Share,Dividend Per Share,Profit Margin,EBITDA,Cash & Cash Equivalents,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Debt/Equity,Quick Ratio,Current Ratio,Dividend Yield,Payout Ratio,Total Return,Close Price
1,1993-09-24,0.041246,0.012333,0.000511,0.000230,-0.033,0.001,0.001436,0.003523,0.066879,...,-0.029925,0.015954,0.035210,0.409745,0.906767,1.730,0.018,0.000,0.018000,0.214504
2,1993-12-31,0.045194,0.011604,0.004962,0.003054,0.026,0.001,0.016201,0.008245,0.085714,...,0.028246,0.000611,-0.006260,0.366372,1.018284,1.807,0.015,0.333,0.015000,0.244714
3,1994-04-01,0.037458,0.010059,0.002627,0.001276,-0.007,0.001,0.008185,0.004954,0.000000,...,-0.001276,0.004804,0.005104,0.000000,0.000000,0.000,0.013,1.000,0.031257,0.195736


In [45]:
reduced_data_val_df = reduce_df_standardize(data_val_df.copy(), columns_to_keep, shares_outstanding_col)
reduced_data_val_df.head(3)

,index,Gross Profit,Research & Development,Operating Income,Net Income,Free Cash Flow Per Share,Dividend Per Share,Profit Margin,EBITDA,Cash & Cash Equivalents,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Debt/Equity,Quick Ratio,Current Ratio,Dividend Yield,Payout Ratio,Total Return,Close Price
0,2017-12-30,1.658166,0.166589,1.284697,0.981101,1.246,0.158,0.227255,1.491773,13.940146,...,1.383419,-0.664499,-0.366770,0.873045,1.105918,1.242,0.015,0.161,0.046946,39.559582
1,2018-03-31,1.165302,0.168064,0.790766,0.687679,0.544,0.158,0.226082,0.980074,13.295151,...,0.752755,1.428393,-1.307097,0.960293,1.235423,1.456,0.015,0.230,0.051717,43.823643
2,2018-06-30,1.045693,0.189516,0.645820,0.589851,0.575,0.182,0.216258,0.860018,12.481292,...,0.741884,0.202113,-1.614191,0.996964,1.099257,1.307,0.014,0.308,0.072640,53.648624


In [46]:
dateStr = datetime.date(datetime.now()).strftime("%d_%m_%Y")
prepared_data_folder = Path(r'C:\Users\candr\Documents\masters\SecurityAnalysisDNN\src\SecurityAnalysisDNN\data\preparedData') / dateStr
if not prepared_data_folder.exists():
    prepared_data_folder.mkdir()

# todo add a column called company so that we can keep track of what the data belongs to.

reduced_data_train_df.to_excel(prepared_data_folder / f'train_{symbol}.xlsx', index=False)
reduced_data_val_df.to_excel(prepared_data_folder / f'val_{symbol}.xlsx', index=False)